**This script generates 500 augmented images from clean image**

In [ ]:
!pip install albumentations


In [ ]:
import albumentations as A
import cv2
import numpy as np
import os

# Load the reference image
image_path = 'Interieur.jpg'
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Load the floor mask and ensure it's of the same size as the image being processed
floormask_path = 'Floormask.png'
floormask_image = cv2.imread(floormask_path, cv2.IMREAD_UNCHANGED)  # Ensure we read the alpha channel
floormask_resized = cv2.resize(floormask_image, (image.shape[1], image.shape[0]))

# Enhanced augmentation pipeline
augment = A.Compose([
    A.RandomBrightnessContrast(p=0.5),
    A.CLAHE(p=0.3),
    A.RandomShadow(shadow_roi=(0, 0.5, 1, 1), num_shadows_lower=1, num_shadows_upper=2, shadow_dimension=5, p=0.5),
    A.GaussianBlur(p=0.3),
    A.MotionBlur(blur_limit=3, p=0.3),
    A.GaussNoise(p=0.3),
    A.ISONoise(p=0.3),
    A.HueSaturationValue(hue_shift_limit=15, sat_shift_limit=30, val_shift_limit=20, p=0.5),
    A.RandomSunFlare(flare_roi=(0, 0, 1, 0.5), angle_lower=0, angle_upper=1, num_flare_circles_lower=3, num_flare_circles_upper=8, p=0.5),
    A.RandomGamma(gamma_limit=(80, 120), p=0.5)
])

# Create an 'output' directory for the augmented images
output_dir = 'output2'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Apply augmentations and save 500 augmented images
for i in range(500):
    augmented_image = augment(image=image)['image']

    # Mask the augmented image using the floor mask
    masked_image = cv2.bitwise_and(augmented_image, augmented_image, mask=floormask_resized[:,:,3])  # Using the alpha channel of the mask

    save_path = os.path.join(output_dir, f"augmented_{i}.jpg")
    cv2.imwrite(save_path, cv2.cvtColor(masked_image, cv2.COLOR_RGB2BGR))


**This script generates 500 dirty and cleann images from clean images**

In [4]:
import cv2
import numpy as np
import os
import random
import albumentations as A
from multiprocessing import Pool, cpu_count

# Load the reference image
image_path = 'Interieur.jpg'
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Load the stain images
stain_paths = [f'vlek{i}.png' for i in range(1, 9)]
stain_examples = [cv2.cvtColor(cv2.imread(stain_path, cv2.IMREAD_UNCHANGED), cv2.COLOR_BGRA2RGBA) for stain_path in stain_paths]

# Load the floor mask
floormask_path = 'Floormask.png'
floormask_image = cv2.imread(floormask_path, cv2.IMREAD_UNCHANGED)
floormask_resized = cv2.resize(floormask_image, (image.shape[1], image.shape[0]))

# Get the coordinates of the mask
y_indices, x_indices = np.where(floormask_resized[:,:,3] > 0)
coordinates = list(zip(y_indices, x_indices))

# Enhanced augmentation pipeline
augment = A.Compose([
    A.RandomBrightnessContrast(p=0.5),
    A.CLAHE(p=0.3),
    A.RandomShadow(shadow_roi=(0, 0.5, 1, 1), num_shadows_lower=1, num_shadows_upper=2, shadow_dimension=5, p=0.5),
    A.GaussianBlur(p=0.3),
    A.MotionBlur(blur_limit=3, p=0.3),
    A.GaussNoise(p=0.3),
    A.ISONoise(p=0.3),
    A.HueSaturationValue(hue_shift_limit=15, sat_shift_limit=30, val_shift_limit=20, p=0.5),
    A.RandomSunFlare(flare_roi=(0, 0, 1, 0.5), angle_lower=0, angle_upper=1, num_flare_circles_lower=3, num_flare_circles_upper=8, p=0.5),
    A.RandomGamma(gamma_limit=(80, 120), p=0.5)
])

def overlay_stains_on_mask(image, stain_examples, coordinates, mask):
    output_image = image.copy()
    for _ in range(random.randint(1, 15)):
        stain = random.choice(stain_examples).copy()

        # Randomly rotate and scale the stain
        rotation_angle = random.randint(0, 360)
        scale_factor = random.uniform(0.8, 1.2)
        M = cv2.getRotationMatrix2D((stain.shape[1] // 2, stain.shape[0] // 2), rotation_angle, scale_factor)
        rotated_stain = cv2.warpAffine(stain, M, (stain.shape[1], stain.shape[0]))

        y, x = random.choice(coordinates)

        # Adjust to top-left corner of the stain
        y = max(y - rotated_stain.shape[0] // 2, 0)
        x = max(x - rotated_stain.shape[1] // 2, 0)

        h, w, _ = rotated_stain.shape

        # Get the overlay area and its dimensions
        overlay_area = output_image[y:y+h, x:x+w]
        overlay_h, overlay_w, _ = overlay_area.shape

        # Adjust the size of the rotated stain and the alpha channel based on the overlay area dimensions
        rotated_stain = rotated_stain[:overlay_h, :overlay_w]
        alpha = rotated_stain[..., 3] / 255.0
        alpha = np.expand_dims(alpha, axis=2)

        blended_area = alpha * rotated_stain[..., :3] + (1 - alpha) * overlay_area

        output_image[y:y+overlay_h, x:x+overlay_w] = blended_area

    # Clear everything outside the floor mask
    output_image = cv2.bitwise_and(output_image, output_image, mask=mask[:,:,3])

    return output_image



# Parallel processing
def generate_image(i):
    augmented_image = augment(image=image)['image']
    masked_image = cv2.bitwise_and(augmented_image, augmented_image, mask=floormask_resized[:,:,3])
    stained_image = overlay_stains_on_mask(masked_image, stain_examples, coordinates, floormask_resized)
    save_path_clean = os.path.join(output_clean_dir, f"clean_{i}.jpg")
    save_path_dirty = os.path.join(output_dirty_dir, f"dirty_{i}.jpg")
    cv2.imwrite(save_path_clean, cv2.cvtColor(masked_image, cv2.COLOR_RGB2BGR))
    cv2.imwrite(save_path_dirty, cv2.cvtColor(stained_image, cv2.COLOR_RGB2BGR))

# Create 'output_clean' and 'output_dirty' directories
output_clean_dir = 'output_clean'
output_dirty_dir = 'output_dirty'
if not os.path.exists(output_clean_dir):
    os.makedirs(output_clean_dir)
if not os.path.exists(output_dirty_dir):
    os.makedirs(output_dirty_dir)

# Use multiprocessing for parallel image generation
with Pool(cpu_count()) as p:
    p.map(generate_image, range(1000))


Download the images

In [5]:
import shutil
import zipfile

# Folders to be downloaded
folders_to_download = ['/content/output_dirty', '/content/output_clean']

# Create a ZIP archive
zip_filename = 'downloaded_folders.zip'
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for folder in folders_to_download:
        folder_name = folder.split('/')[-1]
        for root, dirs, files in os.walk(folder):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, '/content')
                zipf.write(file_path, arcname)

# Download the ZIP archive to your PC
from google.colab import files
files.download(zip_filename)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Remove the images

In [ ]:
import shutil

# List of folder paths to be deleted
folders_to_delete = [
    '/content/output_clean',
    '/content/output_dirty'
]

# Loop through the list and delete each folder
for folder_path in folders_to_delete:
    shutil.rmtree(folder_path)
    print(f"Folder {folder_path} has been deleted.")


Folder /content/output_clean has been deleted.
Folder /content/output_dirty has been deleted.
